
## Building a Conversational Chatbot with Custom Data

This notebook guides you through the creation of a chatbot tailored to your specific data needs. We utilize Elastic Search as our vector storage solution and incorporate the "meta-llama/llama-2-13b-chat" model from WatsonX's Large Language Models (LLM). The `langchain` library plays a crucial role in this process, aiding in tasks like chunking documents, indexing data in Elastic Search, managing conversation chains with memory buffers, and crafting prompt templates.

### Key Features:

- **PDF Content Processing**: When users upload PDF files, the notebook extracts the text, segments it into manageable chunks, and indexes these chunks in Elastic Search using an appropriate `elser` model.
- **Data-Driven Query Handling**: Users can pose questions to the chatbot, which searches the indexed data for relevant answers.
- **Integrating Elastic Search and WatsonX LLMs**: We leverage `langchain`'s capabilities to link Elastic Search indexing with WatsonX's LLMs, enabling a seamless conversational experience with memory and retrieval functionalities.
- **Hallucination Check**: The notebook includes a mechanism to detect and correct any hallucinations or inaccuracies in the LLM's responses.

### Prerequisites for Running the Notebook:


1. **Library Requirements**: Confirm that you have installed all libraries specified in the `requirements.txt` file.
2. **Elastic Search ELSER Model Setup**: Implement an ELSER model within your Elastic Search instance. Refer to the Elastic documentation for setup details: [Elastic Machine Learning: ELSER](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html).
3. **Environment Configuration**: A `.env` file is required, containing critical configuration details:

   - **`elastic_search_url`**: This URL connects you to your Elastic Search instance, a search engine built on the Lucene library, offering distributed, multitenant capabilities for full-text search with a web interface and JSON document handling. The `elastic_search_url` serves as your point of interaction for tasks like data indexing and querying.
   - **`elastic_search_api_key`**: A key for secure access to your Elastic Search instance, this API key is essential for authentication and authorization, ensuring that only permitted users and applications interact with your Elastic Search server.
   - **`WATSONX_APIKEY`**: Your access key for IBM's WatsonX services, this API key is used for authenticating requests to WatsonX's AI and cognitive computing services. Acquire your WatsonX.ai URL and API key by following these instructions: [IBM Cloud: API Keys](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key) and [WatsonX as a Service Documentation](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=library-credentials).
   - **`WATSONX_URL`**: The primary access point for WatsonX's API, this URL is where you connect to utilize WatsonX's diverse AI functionalities.
   - **`WATSONX_Project_ID`**: A unique identifier for your project in the WatsonX environment, this ID helps manage and organize resources like datasets and AI models within your specified WatsonX project.Obtain a project ID for WatsonX AI, essential for project management within WatsonX. Guides for this can be found here: [Creating a WatsonX Project](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=projects-creating-project), [Finding Your Project ID in IBM Cloud](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-project-id.html?context=wx).



Below cell imports the required libraries to run this notebook.

In [1]:
from dotenv import load_dotenv
import os
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import PyPDF2
from langchain.chat_models import ChatOpenAI
from elasticsearch import Elasticsearch
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util



/Users/vikrambhat/Documents/GitHub/RAG-using-elasticsearch-WatsonX/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

### User Inputs 
User can specify/update below inputs as per their needs. 


`es_model_id`: This refers to the unique identifier of the Elastic Search (ES) model that's being used. Elastic Search models, like the ELSER model, are deployed within the Elastic Search environment to perform specific tasks such as text analysis, natural language processing, or vector search. The es_model_id helps in identifying and referencing the specific model deployed in your Elastic Search instance.

`index_name`: In the context of Elastic Search, an index_name denotes the name of the index where your data is stored. The index_name is used to specify which collection of documents you're querying or modifying in your Elastic Search operations. If the index is not already present, one with this name gets created during runtime.

`llm_model_id`: This is the identifier for the Large Language Model (LLM) from WatsonX that you're using. WatsonX provides various AI models, including LLMs for different tasks like conversation, text completion, or language translation. The llm_model_id allows you to specify which of these models you want to interact with in your application.

`wx_url`: This variable represents the URL for the WatsonX service. WatsonX, being a cloud-based service, can be accessed through its dedicated URL. This URL is used to make API requests, authenticate your application, and access the services provided by WatsonX, like their LLMs or other AI functionalities.

`wx_project_id`: The wx_project_id is a unique identifier for a project within the WatsonX ecosystem. In WatsonX, a project is a workspace where you can organize resources, data, models, and other assets. Each project has a unique ID which is used to access and manage the resources within that specific project. This ID ensures that your interactions with the WatsonX API are scoped and managed within the right project context.

In [3]:
load_dotenv()
es_model_id = '.elser_model_2_linux-x86_64'
index_name = "elser_index_vb_test_2"
llm_model_id = "meta-llama/llama-2-13b-chat" #"ibm/granite-13b-chat-v2"#
wx_url = "https://us-south.ml.cloud.ibm.com"
wx_project_id = os.environ["WATSONX_Project_ID"]

### Enter your pdf file name below


In [5]:
pdf_docs=["Industry accelerators - IBM Documentation.pdf"]


### Step 1: Prepare above documents and their metadata
The prepare_docs function below processes a list of PDF documents by extracting text from each page and organizing it into two lists: one for the text content and another for the metadata (titles). It iterates through each page of each PDF, extracts the text, and forms a title using the PDF name and page number. The function returns these two lists, making it useful for indexing and referencing the content of multiple PDFs at a page level.

In [6]:

def prepare_docs(pdf_docs):
    docs = []
    metadata = []
    content = []

    for pdf in pdf_docs:

        pdf_reader = PyPDF2.PdfReader(pdf)
        for index, text in enumerate(pdf_reader.pages):
            doc_page = {'title': pdf + " page " + str(index + 1),
                        'content': pdf_reader.pages[index].extract_text()}
            docs.append(doc_page)
    for doc in docs:
        content.append(doc["content"])
        metadata.append({
            "title": doc["title"]
        })
    print("Content and metadata are extracted from the documents")
    return content, metadata



### Step 2: Chunk the documents 
The get_text_chunks function takes text content and metadata as inputs and splits the content into smaller chunks. It uses a RecursiveCharacterTextSplitter configured with a specified chunk size (512 characters) and overlap (256 characters) for this purpose. The function processes the content, splitting it into passages while maintaining associated metadata. After splitting, it prints the total number of passages created and returns these split documents. This function is useful for breaking down large text into more manageable, indexed segments for easier processing and retrieval.

In [7]:
def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,
        chunk_overlap=256,
    )
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs


### Step 3: Ingest into Elastic Search 
The ingest_and_get_vector_store function initializes and populates an Elasticsearch vector store with provided document chunks (split_docs). It creates an ElasticsearchStore instance using environment variables for the Elastic Search URL, API key, index name, and a retrieval strategy based on a specified Elastic Search model ID (es_model_id). The function then ingests the split documents into this Elasticsearch store. After processing, it returns the populated vector_store, enabling the storage and retrieval of document vectors for efficient search and analysis. This function essentially sets up and populates an Elasticsearch-based vector store tailored for handling segmented document data.

In [8]:
def ingest_and_get_vector_store(split_docs):
    vector_store = ElasticsearchStore(
                    es_cloud_id= os.environ["elastic_search_cloud_id"],
                    es_api_key=os.environ["elastic_search_api_key"],
                    index_name=index_name,
                    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=es_model_id)
                    )
    documents = vector_store.from_documents(
        split_docs,
        es_cloud_id= os.environ["elastic_search_cloud_id"],
        es_api_key=os.environ["elastic_search_api_key"],
        index_name=index_name,
        strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=es_model_id)
    )
    print("Documents indexed and vector Store returned")

    return vector_store


### Step 4: Set up Conversation Chain using LLM
The `get_conversation_chain` function sets up a conversational chain for a chatbot using a vector store, a language model, and memory management. Key steps include:

1. **Setting Up LLM Parameters**: Defines various generation parameters for the WatsonX Large Language Model (LLM), like decoding method, token limits, temperature, and selection criteria (Top-K, Top-P).

2. **Initializing WatsonX LLM**: Creates a `WatsonxLLM` instance using the LLM model ID, WatsonX URL, project ID, the specified parameters, and an API key from the environment.

3. **Creating a Retriever**: Transforms the provided `vector_store` into a retriever for fetching relevant documents.

4. **Preparing a Prompt Template**: Utilizes a prompt template for structuring the queries sent to the LLM. User can create/update their own template below.

5. **Setting Up Conversation Memory**: Implements a `ConversationBufferMemory` to manage chat history and output answers.

6. **Building the Conversational Chain**: Constructs a `ConversationalRetrievalChain` by combining the LLM, retriever, prompt template, and memory. This chain also returns source documents alongside responses.

The function ultimately returns this configured conversation chain, which is essential for handling and responding to user queries effectively in a chatbot, integrating both generative AI and information retrieval capabilities.

In [16]:
template ="""[INST]You are a helpful, respectful, and honest assistant. 
Always answer as helpfully as possible, while being safe. Be brief in your answers. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.If you don\\'\''t know the answer to a question, please do not share false information. \n Answer with no more than 150 words, in 2 or 3 sentences. If you cannot base your answer on the given document, please state that you do not have an answer.\n\n{question} Answer with no more than 200 words. If you cannot base your answer on the given document, please state that you do not have an answer. do not include a question in your response. dont prompt to make select correct answers[/INST]"""

template = """[INST]
As an AI, provide accurate and relevant information based on the provided document. Your responses should adhere to the following guidelines:
- Answer the question based on the provided documents.
- Be direct and factual, limited to 50 words and 2-3 sentences. Begin your response without using introductory phrases like yes, no etc.
- Maintain an ethical and unbiased tone, avoiding harmful or offensive content.
- If the document does not contain relevant information, state "I cannot provide an answer based on the provided document."
- Avoid using confirmatory phrases like "Yes, you are correct" or any similar validation in your responses.
- Do not fabricate information or include questions in your responses.
- do not prompt to select answers. do not ask me questions

{question}


[/INST]
"""

#template = """Given the document and the current conversation between a user and an agent, your task is as follows: Answer any user query by using information from the document. The response should be detailed."""



def get_conversation_chain(vector_store):
    openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo")
    retriever = vector_store.as_retriever()
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

    conversation_chain = (ConversationalRetrievalChain.from_llm
                          (llm=openai_llm,
                           retriever=retriever,
                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                           memory=memory,
                           return_source_documents=True))
    print("Conversational Chain created for the LLM using the vector store")
    return conversation_chain


### Step 5: Detect Hallucination in the LLMs Response
The `validate_answer_against_sources` function evaluates the reliability of a response by comparing it with source documents. It works as follows:

1. **Model Initialization**: Utilizes the SentenceTransformer model 'all-MiniLM-L6-v2' to generate embeddings.

2. **Threshold Setting**: Sets a similarity threshold (here, 0.5) to determine the acceptable level of similarity between the response and source documents.

3. **Extracting Source Texts**: Gathers the content of the source documents.

4. **Computing Embeddings**: Generates embeddings for both the response answer and the source texts.

5. **Calculating Similarity**: Computes cosine similarity scores between the response answer's embedding and the embeddings of each source text.

6. **Validity Check**: Checks if any of the similarity scores exceed the set threshold. If yes, it implies that the response is sufficiently similar to at least one of the source documents, suggesting its reliability, and returns `True`. If not, it returns `False`.

Essentially, this function serves as a mechanism to check the alignment of the chatbot's response with the information in the source documents, ensuring the response's accuracy and relevance.

In [10]:

def validate_answer_against_sources(response_answer, source_documents):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    similarity_threshold = 0.5  
    source_texts = [doc.page_content for doc in source_documents]

    answer_embedding = model.encode(response_answer, convert_to_tensor=True)
    source_embeddings = model.encode(source_texts, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(answer_embedding, source_embeddings)


    if any(score.item() > similarity_threshold for score in cosine_scores[0]):
        return True  

    return False  


Now that we have crafted all the necessary functions, it's time to put them into action and test their functionality.

In [11]:
content, metadata = prepare_docs(pdf_docs)


Content and metadata are extracted from the documents


In [12]:
split_docs = get_text_chunks(content, metadata)

Documents are split into 3 passages


In [13]:
vectorstore = ingest_and_get_vector_store(split_docs)

Documents indexed and vector Store returned


In [17]:
conversation_chain=get_conversation_chain(vectorstore)

Conversational Chain created for the LLM using the vector store


### Ask your Question

We created a conversational chain and now ready to chat with your own data. 


### Question 1

In [27]:
user_question = "what are industry accelerators?"
response=conversation_chain({"question": user_question})
print("Q: ",user_question)
print("A: ",response['answer'])

Q:  what are industry accelerators?
A:  I'm sorry, but that is not the correct definition of industry accelerators in the context of IBM Documentation. According to the provided information, industry accelerators are a set of artifacts provided by IBM that help address common business issues. They are designed to help solve specific business problems and typically include a sample project with instructions, data sets, notebooks, and models. They also include a business glossary that provides terms and categories for data governance.


We have now received an answer for a provided question. We can also view the conversation history and source documents in the response.


### Question 2

In [28]:
user_question = "where are they located?"
response=conversation_chain({"question": user_question})
print("Q: ",user_question)
print("A: ",response['answer'])
print("\nConversation Chain: \n",response)

Q:  where are they located?
A:  I'm sorry, but I don't have enough information to provide an answer based on the provided document.

Conversation Chain: 
 {'question': 'where are they located?', 'chat_history': [HumanMessage(content='what are industry accelerators?'), AIMessage(content='Industry accelerators are a set of artifacts provided by IBM that help address common business issues. Each accelerator is designed to solve a specific business problem, such as preventing credit card fraud or optimizing the efficiency of a contact center. These accelerators include sample projects with instructions, data sets, Jupyter notebooks, models, and R Shiny applications. They also include a business glossary that consists of terms and categories for data governance. Industry accelerators are intended for data scientists or business analysts who analyze data and build models to solve business problems.'), HumanMessage(content='where can i find them?'), AIMessage(content="I'm sorry, but I don't h

In [24]:
response['source_documents']

[Document(page_content="01/01/2024, 17:50 Industry accelerators - IBM Documentation\nhttps://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x?topic=integrations-industry-accelerators 3/3A set of business terms to describe data, with logical relationships between terms. See Business\nterms.\nA category with the same name as the accelerator in which to organize the terms. See\nCategories.\nRequirements\nThe IBM® Knowledge Catalog service.\nYou must have the Manage governance categories permission. To see which permissions you\nhave, click your user avatar, select Profile and settings, and then view the Permissions page. If\nyou need more permissions, contact your Cloud Pak for Data administrator.\nProcess overview\nEach accelerator provides detailed instructions that you can access after you download the\naccelerator. These general steps provide an overview of the process:\n1. Import the category.\n2. Import the business terms.\n3. Publish the business terms so that they are available in all

### Detect and Solve Hallucinations

The response to the second question appears to be generated by the LLMs and not directly retrieved from the documents, resulting in an answer that seems out of context. To address such instances of misinformation or 'hallucination,' we previously developed the function `validate_answer_against_sources`. We can use this function to cross-check the answer with the source documents to ensure its accuracy and relevance.

In [141]:
if response['source_documents']:
    response_answer = response['answer']
    source_docs = response['source_documents']

    # Post-processing step to validate the answer against the source documents
    is_valid_answer = validate_answer_against_sources(response_answer, source_docs)
    if not is_valid_answer:
        response['answer'] = "Sorry I can not answer the question based on the given documents"
else:
    response['answer'] ="Sorry, I cannot answer the question based on the given documents"

print("Q: ",user_question)
print("A: ",response['answer'])

Q:  who developed them?
A:  Sorry I can not answer the question based on the given documents


We have now set up end to end Retrieval Augmented Generation Chatbot using Elastic Search, LangChain and WatsonX. 